In [1]:
from urllib.request import urlretrieve
import jieba
import warnings
warnings.filterwarnings("ignore")
"""
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.big")
url = "https://github.com/Elwing-Chou/mtkdl/raw/master/ptt_dic.txt"
urlretrieve(url, "dict.ptt")
"""
jieba.set_dictionary("dict.big")
jieba.load_userdict("dict.ptt")

Building prefix dict from D:\onedrive\Documents\data\dict.big ...
Loading model from cache R:\TEMP\jieba.u3fc4963e5b0b71d0127ab7586a4086cb.cache
Loading model cost 1.424 seconds.
Prefix dict has been built successfully.


In [2]:
import pandas as pd
data = pd.read_csv("data.csv", header=0)
data.head()[:1]

,產品名,分數,產品圖片,品牌,分類,評論1,評論2,評論3,評論4,評論5,評論6,評論7,評論8,評論9,評論10
0,三色修容餅 Triple Contour Palette,5.4,https://dg9ugnb21lig7.cloudfront.net/uploads/p...,heme,修容餅,Heme三色修容盤 已經購入一陣子啦～ 用到鐵皮君出沒😆 色號：01 NATIONA...,獲得超多美妝YouTuber 推薦的 heme三色修容餅 以前化妝沒那麼注重修容 都隨便刷刷...,這款修容上市前 我一直都是使用 too cool for school 的修容餅 不得不說h...,"以前沒有使用修容的習慣,後來開始上課學習彩妝以後 經過老師的說明,還有一些美妝YT的影片後 ...",這塊修容餅粉質綿密是屬於沒那麼顯色的，但就是因為不太顯色的特性，很適合修容或製造眼窩及鼻影...,官網表示: 首次推出「三色修容餅」，精選冷暖兩大色調，特調三色色塊圓型均分，可任意沾取混色，...,在IG上抽獎抽到的 顏色還不錯 可惜用沒兩次就不小心摔碎了QQ 沒辦法提供太多心得 只能挖I...,首先要先說他的價格，購入價格是299真的cp很高，對於之前沒用過修容的我也很容易上手，我是用...,買了00號冷色調 最淺色用作眼窩打底 中間色打鼻影兩側 中間跟最深色混刷打在下顎線 覺得很自...,NONE


In [3]:
df = data.copy()
df["combine"] = df[df.columns[5:]].apply(" ".join, axis=1)

In [4]:
import re
def process(content):
    content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
    punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')
    cut = filter(lambda x: x not in punct, jieba.cut(content))
    return " ".join(cut)
df["combine"] = df["combine"].apply(process)
df["combine"] = df["combine"].str.replace("NONE", "")
df["combine"] = df["combine"].str.replace("的", "")
df["combine"] = df["combine"].str.replace("我", "")
df["combine"] = df["combine"].str.replace("是", "")
df["combine"] = df["combine"].str.replace("買", "")
df["combine"] = df["combine"].str.replace("了", "")
df["combine"] = df["combine"].str.replace("有", "")
df["combine"] = df["combine"].str.replace("很", "")

In [5]:
with open("lines.txt", "w", encoding="utf-8") as f:
    for content in df["combine"]:
        f.write(content)
        f.write("\n")

In [6]:
from gensim.models.word2vec import LineSentence
from gensim.models.word2vec import Word2Vec
sentences = LineSentence("lines.txt")
model = Word2Vec(sentences)

In [7]:
vocab = "武士刀"
model.wv[vocab]
model.wv.most_similar(vocab)

[('好久沒', 0.9982296824455261),
 ('時期', 0.998132586479187),
 ('D', 0.9981110692024231),
 ('先前', 0.9977195262908936),
 ('這些', 0.9976620674133301),
 ('經驗', 0.9976290464401245),
 ('聽到', 0.9976266622543335),
 ('鐵片', 0.9974688291549683),
 ('曾經', 0.9973751306533813),
 ('ysl', 0.9973300099372864)]

In [8]:
import numpy as np
import jieba.analyse
encodings = []
for content in df["combine"]:
    tags = jieba.analyse.extract_tags(
        content, 
        topK=None, 
        withWeight=True
    )
    total = np.zeros((100, ))
    weight = 0.0
    for word, tfidf in tags:
        if word in model.wv.vocab:
            total = total + tfidf * model.wv[word]
            weight = weight + tfidf
    encodings.append(total / weight)
print(len(encodings))

867


In [9]:
with open("w2v_vocab.wv", "w", encoding="utf-8") as f:
    f.write("{} {}\n".format(867, 100))
    for i, wv in enumerate(encodings):
        wv = " ".join(map(str, wv))
        f.write("{} {}\n".format(i, wv))

In [10]:
from gensim.models import KeyedVectors
d2v = KeyedVectors.load_word2vec_format("w2v_vocab.wv")

In [13]:
tok = input("Enter(0-866):")
data = df.iloc[int(tok)]
print("品牌：", data["品牌"])
print("分數：", data["分數"])
print("產品名：", data["產品名"])
print("內容：", data["combine"])
print("-" * 30)
tok2, sim = d2v.most_similar(tok)[0]
print("相似度:", sim)
print("-" * 30)
data = df.iloc[int(tok2)]
print("品牌：", data["品牌"])
print("分數：", data["分數"])
print("產品名：", data["產品名"])
print("內容：", data["combine"])

Enter(0-866):5
品牌： 3 Concept Eyes
分數： 3.9
產品名： GLOW BEAM HIGHLIGHTER
內容： 被 幾位 youtuber 跟 網友 文章 燒到 之前 剛好  折扣 就 來 試 看看 這款 它  氣墊 打亮  特別 使用 上  方便 膏體 也  好 取用 打亮 上 在 臉上   自然  亮度    色號  ＃ 𝑹 𝑬 𝑨 𝑹 𝑺 𝑰 𝑫 𝑬 膏 體質 地  濕潤 油滑 軟軟  外包裝  塑膠 尺寸 蠻 小 攜帶方便 ⇡ 自然光 上 跟 室內 光 下 ⇡ 氣墊 材質  黏性  好 看 YOUTUBE 整個 被 查理 燒到 一直 以來 都 追求 光澤 肌  妝 感 用  多 打亮 都 讓  失望 因為 都 太 不 自然 不 粉感 重 就 亮片 大 顯 毛孔  擁 大約 5 個 打亮 沒 一個 讓  滿意 直到 這個 打亮  出現 讓  知道 從 肌膚 亮 出來  自然 光澤  什麼 選  色號  跟 查理 一樣  # REAR SIDE 打開 裡面   鏡子         
------------------------------
相似度: 0.9980628490447998
------------------------------
品牌： CHANEL 香奈兒
分數： 5.9
產品名： 超炫耀釉光唇萃
內容： 大家 晚安 先 預祝 各位 連假 愉快 這次 想來 分享 新 上市  唇萃 搭配 限定  木質 禮盒 總共  6 色 內文 圖多 剛好  配合 活動 + 預約 禮物 所以 看起來  澎湃 😂 1 左邊  裸色 毛毛 包 12000 滿額 禮 2 餅乾  部分  預約 禮 3 裸色 紙盒  新 上市 水潤 粉底  禮物 4 右下  活動  香氛 書籤 拿到 遠百禮 卷 本來 想 去 個 果凍 氣墊 但蕊心 都 沒貨 香奈兒 可以 不要 這樣 嗎 言歸正傳 正巧 看到 Vogue Korea 少女時代 Tiffany  what s in my bag 她 推薦  這個 64 號色 這款 唇 釉剛 上市 時  就 去 試過 但色 號太多 眼花撩亂 選不出 什麼 名堂 就 轉身 離開  如今 再 看到 T 原本  想 購入 74 号色 但 已經 賣完  所以   備選 色 72 号色 顏色 比較 透 一些 

In [17]:
target_data = pd.read_csv("target.csv", header=0)
target_df = target_data.copy()
target_data

,title,content
0,nature自然妝感,你上一次化全妝是什麼時候呢？很多人的答案會是一多個月以前。2021春夏時尚週妝容多以「自然」...
1,wild狂野風格,Hermès 一改以往的裸妝路線，今年首度以自家的口紅亮相時裝週，化妝師Dick Page使...
2,smoke深黑眼線,DIOR秀上模特立體深邃的雙眸，迪奧彩妝創意形象總監Peter Philips表示：「我想以...
3,nature鮮明頰紅,將柔嫩的粉色腮紅大面積暈染兩頰，和模特兒的深棕膚色和奶油色裸唇形成了鮮明的對比，既清新又俏麗...


In [18]:
import numpy as np
import jieba.analyse
encodings = []
for content in target_df["content"]:
    tags = jieba.analyse.extract_tags(
        content, 
        topK=None, 
        withWeight=True
    )
    total = np.zeros((100, ))
    weight = 0.0
    for word, tfidf in tags:
        if word in model.wv.vocab:
            total = total + tfidf * model.wv[word]
            weight = weight + tfidf
    encodings.append(total / weight)
print(len(encodings))

4


In [19]:
with open("target_w2v_vocab.wv", "w", encoding="utf-8") as f:
    f.write("{} {}\n".format(4, 100))
    for i, wv in enumerate(encodings):
        wv = " ".join(map(str, wv))
        f.write("{} {}\n".format(i, wv))

In [20]:
from gensim.models import KeyedVectors
target_d2v = KeyedVectors.load_word2vec_format("target_w2v_vocab.wv")

In [ ]:
tok = input("Enter(0-3):")
data = target_df.iloc[int(tok)]
print("風格：", data["title"])
print("內容：", data["content"])
print("-" * 30)
tok2, sim = d2v.most_similar(tok)[0]
print("相似度:", sim)
print("-" * 30)
data = df.iloc[int(tok2)]
print("品牌：", data["品牌"])
print("分數：", data["分數"])
print("產品名：", data["產品名"])
print("內容：", data["combine"])